In [19]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

MODEL_NUMBER = 6

    
# seed so random stuff isnt random
SEED = 100

# transforms with imagenet normalization
transform = transforms.Compose([
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

dataset = torchvision.datasets.EuroSAT(root='./data', download=True, transform=transform)

# split into train, test
# 80% train, 20% test 
train_ratio = 0.8
test_ratio = 0.2


total_size = len(dataset)
train_size = int(total_size * train_ratio)
test_size = total_size - train_size


train_dataset, test_dataset = random_split(
    dataset, [train_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"train: {len(train_dataset)}")
print(f"test size: {len(test_dataset)}")

train: 21600
test size: 5400


In [20]:
import torchvision.models as models

# resnet 18 model pretrained on imagenet
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)


# classifier head with 10 outputs
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)

print(f"Model: ResNet-18 (ImageNet pretrained)")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")


Model: ResNet-18 (ImageNet pretrained)
Total parameters: 11181642


In [21]:

# dataloaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=128, 
    shuffle=True,
    pin_memory=True, # for use on gpu
    num_workers=4 # arbitrarily chosen. some sources recommend 4 per gpu
)


test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    pin_memory=True,
    num_workers=4
)

In [22]:

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    model.to(device, non_blocking=True) # according to pytorch docs we can do this if memory is pinned
else:  
    model.to(device)
    


In [23]:
def train_one_epoch():
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def test_one_epoch():
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [ ]:
# tensorboard setup
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/train_{}_{}'.format(MODEL_NUMBER, timestamp))

checkpoint_dir = f'checkpoints/train_{MODEL_NUMBER}_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)

EPOCHS = 2000

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    


    train_loss, train_acc = train_one_epoch()
    
    test_loss, test_acc = test_one_epoch()
    
    print(f'train loss: {train_loss:.4f} | train acc: {train_acc:.2f}%')
    print(f'test loss:  {test_loss:.4f} | test acc:  {test_acc:.2f}%')
    
    # log data
    writer.add_scalar('loss/1_train', train_loss, epoch + 1)
    writer.add_scalar('loss/2_test', test_loss, epoch + 1)
    writer.add_scalar('accuracy/1_train', train_acc, epoch + 1)
    writer.add_scalar('accuracy/2_test', test_acc, epoch + 1)
    writer.flush()
    
    # checkpoint every 20 epochs
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f'checkpoint saved: {checkpoint_path}')
    
    print()

# save final model
final_checkpoint_path = os.path.join(checkpoint_dir, f'model_final.pt')
torch.save(model.state_dict(), final_checkpoint_path)
print(f'final checkpoint saved: {final_checkpoint_path}')

writer.close()
print('Complete')

Epoch 1/2000
train loss: 0.0222 | train acc: 99.24%
test loss:  0.1760 | test acc:  95.48%

Epoch 179/2000
train loss: 0.0227 | train acc: 99.19%
test loss:  0.2145 | test acc:  94.80%

Epoch 180/2000
train loss: 0.0294 | train acc: 99.03%
test loss:  0.2104 | test acc:  94.44%
checkpoint saved: checkpoints/train_6_20251119_222909/model_epoch_180.pt

Epoch 181/2000
train loss: 0.0220 | train acc: 99.25%
test loss:  0.2341 | test acc:  94.26%

Epoch 182/2000
train loss: 0.0264 | train acc: 99.12%
test loss:  0.1860 | test acc:  95.04%

Epoch 183/2000
train loss: 0.0196 | train acc: 99.33%
test loss:  0.1884 | test acc:  94.98%

Epoch 184/2000
train loss: 0.0275 | train acc: 99.18%
test loss:  2.2177 | test acc:  87.65%

Epoch 185/2000
train loss: 0.0782 | train acc: 97.93%
test loss:  0.2056 | test acc:  94.06%

Epoch 186/2000
train loss: 0.0263 | train acc: 99.13%
test loss:  0.2245 | test acc:  94.26%

Epoch 187/2000
train loss: 0.0245 | train acc: 99.15%
test loss:  0.2117 | test acc

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8be54559e0>Exception ignored in: 
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8be54559e0><function _MultiProcessingDataLoaderIter.__del__ at 0x7f8be54559e0>Traceback (most recent call last):


  File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
Traceback (most recent call last):
Traceback (most recent call last):
      File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
  File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()    
self._shutdown_workers()self._shutdown_workers()  File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers


      File "/home/noah/miniconda3/envs/m

train loss: 0.0023 | train acc: 99.91%
test loss:  0.2934 | test acc:  95.24%

Epoch 1940/2000


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8be54559e0>
Traceback (most recent call last):
  File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/home/noah/miniconda3/envs/ml/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
     Exception ignored in:  <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8be54559e0> 
 Traceback (most recent call last):
   File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
      self._shutdown_workers()^
^  File "/home/noah/miniconda3/envs/ml/lib/python3.11/site-packages/torch/utils/dat

train loss: 0.0021 | train acc: 99.93%
test loss:  0.2843 | test acc:  95.33%
checkpoint saved: checkpoints/train_6_20251119_222909/model_epoch_1940.pt

Epoch 1941/2000
train loss: 0.0020 | train acc: 99.94%
test loss:  0.2908 | test acc:  95.31%

Epoch 1942/2000
train loss: 0.0034 | train acc: 99.89%
test loss:  0.3124 | test acc:  94.26%

Epoch 1943/2000
train loss: 0.0026 | train acc: 99.92%
test loss:  0.2823 | test acc:  95.28%

Epoch 1944/2000
train loss: 0.0013 | train acc: 99.93%
test loss:  0.2846 | test acc:  95.54%

Epoch 1945/2000
train loss: 0.0025 | train acc: 99.92%
test loss:  0.3208 | test acc:  95.26%

Epoch 1946/2000
train loss: 0.0015 | train acc: 99.94%
test loss:  0.3185 | test acc:  95.09%

Epoch 1947/2000
train loss: 0.0024 | train acc: 99.94%
test loss:  0.2713 | test acc:  95.56%

Epoch 1948/2000
train loss: 0.0036 | train acc: 99.88%
test loss:  0.2877 | test acc:  95.22%

Epoch 1949/2000
train loss: 0.0015 | train acc: 99.94%
test loss:  0.3143 | test acc:  